# QGAR - A flashcard generating NLP model

\<Description goes here\>

**Table of Contents:**
1. [Download and Preprocess SQuAD Dataset](#download-and-preprocess-squad-dataset)
2. [Run QGAR](#run-qgar)

## Download and Preprocess SQuAD Dataset

First, we download and preprocess the modified `SQuAD` dataset, adding separator (\<sep\>) and end of sequence tokens (\<\s\>) to each entry.
The preprocessed file is split in two datasets, `training` and `validation`, and the sets are saved in the `data` directory in `PyTorch` format under the names:
* [training_data.pt](./data/training_data.pt)
* [validation_data.pt](./data/validation_data.pt)

In [1]:
from preprocessing.preprocessor import Preprocessor
preprocessor = Preprocessor("t5-base")
preprocessor.preprocess_dataset()

2023-03-10 13:56:15.746999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/laugu/miniconda3/envs/hugging/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To 

  0%|          | 0/2 [00:00<?, ?it/s]

Download complete.
Loading cached processed dataset at /home/laugu/.cache/huggingface/datasets/squad_processor/plain_text/1.0.0/5bede5a6c2624a8b962d3568a4e7b4c52fcca7b4dbe53505d5f1b200e89db3d8/cache-805eb09537e94a77.arrow
Loading cached processed dataset at /home/laugu/.cache/huggingface/datasets/squad_processor/plain_text/1.0.0/5bede5a6c2624a8b962d3568a4e7b4c52fcca7b4dbe53505d5f1b200e89db3d8/cache-ae1776303accdf7b.arrow
Loading cached processed dataset at /home/laugu/.cache/huggingface/datasets/squad_processor/plain_text/1.0.0/5bede5a6c2624a8b962d3568a4e7b4c52fcca7b4dbe53505d5f1b200e89db3d8/cache-7e600a8fe2e15b48.arrow
Loading cached processed dataset at /home/laugu/.cache/huggingface/datasets/squad_processor/plain_text/1.0.0/5bede5a6c2624a8b962d3568a4e7b4c52fcca7b4dbe53505d5f1b200e89db3d8/cache-22d39e878cf8c3d0.arrow
Loading cached processed dataset at /home/laugu/.cache/huggingface/datasets/squad_processor/plain_text/1.0.0/5bede5a6c2624a8b962d3568a4e7b4c52fcca7b4dbe53505d5f1b200e89d

## Run QGAR

In [2]:
from pipelines import pipeline
from main import main
main()

{
	"model_name": "t5-base",
	"model_type": "t5",
	"training_file_path": "data/training_data.pt",
	"validation_file_path": "data/validation_data.pt",
	"output_dir": "./models/"
}



/home/laugu/miniconda3/envs/hugging/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/home/laugu/miniconda3/envs/hugging/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch impl

TypeError: string indices must be integers

In [2]:
nlp = pipeline()
nlp("Historical Fiction is one of those sub-genres of literature that takes many forms. It's most important feature, though, is that it's set in the past, with every element of the story conforming to the norms of the day. Here's how we define Historical Fiction, a look at its origins, and some popular types.")